### Presentation Types:  Fuga, ID, and PEN

* **Updated June 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)

In [12]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### Load the Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [14]:
prefix = 'https://crimproject.org/mei/'
# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0038.mei'
url = prefix + mei_file

piece = importScore(url)


print(piece.metadata)


Successfully imported https://crimproject.org/mei/CRIM_Model_0038.mei
{'title': 'Ultimi miei sospiri', 'composer': 'Philippe Verdelot'}


## Find Presentation Types
* `piece.presentationTypes()`

- set the length of the soggetti with `melodic_ngram_length = n`
- set the maximum difference between similar soggetti with `edit_distance_threshold = n`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`

* For example:

`piece.presentationTypes(edit_distance_threshold = 0,
                        include_hidden_types = False,
                        combine_unisons = True,
                       melodic_ngram_length = 4)`

In [17]:

piece.presentationTypes(edit_distance_threshold = 1,
                        include_hidden_types = True,
                        combine_unisons = True,
                       melodic_ngram_length = 4)


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Philippe Verdelot,Ultimi miei sospiri,0.0,"[1/1.0, 5/1.0]",[P-4],"[0.0, 32.0]","[(3, -2, -3, 2), (2, -2, -3, 2)]",[32.0],"[Cantus, Quintus]",FUGA,2,True
1,Philippe Verdelot,Ultimi miei sospiri,0.0,"[1/1.0, 2/1.0, 4/3.0, 5/1.0]","[P-8, P-4, P8, P-12]","[0.0, 8.0, 28.0, 32.0, 32.0]","[(3, -2, -3, 2), (5, -2, -3, 2), (4, -2, -3, 2...","[8.0, 20.0, 4.0, 0.0]","[Cantus, Tenor, Sextus, Quintus, Bassus]",FUGA,5,True
2,Philippe Verdelot,Ultimi miei sospiri,0.0,"[1/1.0, 4/3.0, 5/1.0]","[P-11, P8]","[0.0, 28.0, 32.0]","[(3, -2, -3, 2), (4, -2, -3, 2), (2, -2, -3, 2)]","[28.0, 4.0]","[Cantus, Sextus, Quintus]",FUGA,3,True
3,Philippe Verdelot,Ultimi miei sospiri,8.0,"[2/1.0, 4/3.0, 5/1.0]","[P-4, P-5]","[8.0, 28.0, 32.0]","[(5, -2, -3, 2), (4, -2, -3, 2)]","[20.0, 4.0]","[Tenor, Sextus, Bassus]",FUGA,3,True
4,Philippe Verdelot,Ultimi miei sospiri,62.0,"[8/4.0, 9/3.0, 12/2.0, 12/3.0, 13/1.0]","[P5, P-5, P-4, P8]","[62.0, 68.0, 90.0, 92.0, 96.0]","[(5, -2, -2, -2), (4, -2, -2, -2)]","[6.0, 22.0, 2.0, 4.0]","[Sextus, Altus, Tenor, Bassus, Quintus]",FUGA,5,True
5,Philippe Verdelot,Ultimi miei sospiri,62.0,"[8/4.0, 9/3.0, 12/2.0, 13/1.0]","[P5, P-5, P5]","[62.0, 68.0, 90.0, 96.0]","[(5, -2, -2, -2), (4, -2, -2, -2)]","[6.0, 22.0, 6.0]","[Sextus, Altus, Tenor, Quintus]",ID,4,True
6,Philippe Verdelot,Ultimi miei sospiri,124.0,"[16/3.0, 20/1.0]","[M-3, m-3]","[124.0, 124.0, 152.0]","[(3, -2, -3, 2), (2, -2, -3, 2)]","[0.0, 28.0]","[Cantus, Altus, Quintus]",FUGA,3,True
7,Philippe Verdelot,Ultimi miei sospiri,124.0,"[16/3.0, 19/3.0, 20/1.0]","[M-3, m-10, P8]","[124.0, 124.0, 148.0, 152.0]","[(3, -2, -3, 2), (4, -2, -3, 2), (2, -2, -3, 2)]","[0.0, 24.0, 4.0]","[Cantus, Altus, Sextus, Quintus]",FUGA,4,True
8,Philippe Verdelot,Ultimi miei sospiri,128.0,"[17/1.0, 19/3.0, 20/1.0]","[P-4, P-5]","[128.0, 148.0, 152.0]","[(5, -2, -3, 2), (4, -2, -3, 2)]","[20.0, 4.0]","[Tenor, Sextus, Bassus]",FUGA,3,True
9,Philippe Verdelot,Ultimi miei sospiri,128.0,"[17/1.0, 19/3.0, 20/1.0]","[P-4, P8, P-12]","[128.0, 148.0, 152.0, 152.0]","[(3, -2, -3, 2), (5, -2, -3, 2), (4, -2, -3, 2...","[20.0, 4.0, 0.0]","[Tenor, Sextus, Quintus, Bassus]",FUGA,4,True


## Find Presentation Types in Several Pieces at Once

* Results are combined into a single dataframe
* Set thresholds and other features as noted above

In [9]:

prefix = 'https://crimproject.org/mei/'

piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [11]:

final = pd.DataFrame()
for work in piece_list:
    url = prefix + work
    piece = importScore(url)
    output = piece.presentationTypes(edit_distance_threshold = 0,
                        include_hidden_types = False,
                        combine_unisons = True,
                       melodic_ngram_length = 4)
    
    final = final.append(output, ignore_index=True)
final.head()

Successfully imported https://crimproject.org/mei/CRIM_Mass_0005_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0005_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0005_5.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0001.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0002_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0002_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0002_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0002_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0002_5.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0015.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0013_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0013_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0013_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0013_4.mei
Successf

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Antoine de Févin,Missa Ave Maria: Credo,0.0,"[1/1.0, 3/3.0]",[P8],"[0.0, 20.0]","[(4, 2, 2, -3)]",[20.0],"[Altus, Sup[erius]]",FUGA,2,False
1,Antoine de Févin,Missa Ave Maria: Credo,30.0,"[4/4.0, 7/2.0]",[P8],"[30.0, 50.0]","[(4, -2, -2, 2)]",[20.0],"[Altus, Sup[erius]]",FUGA,2,False
2,Antoine de Févin,Missa Ave Maria: Credo,118.0,"[15/4.0, 16/2.0]",[P5],"[118.0, 122.0]","[(2, 2, 2, 2)]",[4.0],"[Altus, Sup[erius]]",FUGA,2,False
3,Antoine de Févin,Missa Ave Maria: Credo,190.0,"[24/4.0, 25/2.0]",[P5],"[190.0, 194.0]","[(-2, -3, 3, -2)]",[4.0],"[Bassus, Tenor]",FUGA,2,False
4,Antoine de Févin,Missa Ave Maria: Credo,210.0,"[27/2.0, 27/4.0]",[P5],"[210.0, 214.0]","[(2, 2, 2, -2)]",[4.0],"[Bassus, Tenor]",FUGA,2,False


#### Save to your folder of CSV's here in the Jupyter Hub
#### You can save as CSV, or as Excel.
#### You will then need to download this to your computer to view it properly

* Note that in the following part of the code below, you will need to give your file a name:

`saved_csv/**file_name**.xlsx`

In [26]:
writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')
points.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [6]:
points.to_csv('saved_csv/your_file_title.csv')